# Setting

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

from tqdm import tqdm, trange

import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
class CFG:
    #REF_PATH = '/kaggle/input/icr-identify-age-related-conditions/'
    REF_PATH = './data/'
        
    ADD_INTERACTION = False
    FS_ALPHA = 0.01
    
    SEED = 0
    N_SPLITS = 5
    ES = 300
    
    N_TRIALS = 50
    N_JOBS = 1

<br></br>

# Data Load

In [4]:
train_df  = pd.read_csv(CFG.REF_PATH+'train.csv')
test_df   = pd.read_csv(CFG.REF_PATH+'test.csv')
greeks_df = pd.read_csv(CFG.REF_PATH+'greeks.csv')

In [5]:
train_df.columns  = [col.replace(' ','') for col in train_df.columns]
test_df.columns   = [col.replace(' ','') for col in test_df.columns]
greeks_df.columns = [col.replace(' ','') for col in greeks_df.columns]

In [6]:
train_df.shape, greeks_df.shape

((617, 58), (617, 6))

In [7]:
train_df.head()

,Id,AB,AF,AH,AM,AR,AX,AY,AZ,BC,BD,BN,BP,BQ,BR,BZ,CB,CC,CD,CF,CH,CL,CR,CS,CU,CW,DA,DE,DF,DH,DI,DL,DN,DU,DV,DY,EB,EE,EG,EH,EJ,EL,EP,EU,FC,FD,FE,FI,FL,FR,FS,GB,GE,GF,GH,GI,GL,Class
0,000ff2bfdfe9,0.209377,3109.03329,85.200147,22.394407,8.138688,0.699861,0.025578,9.812214,5.555634,4126.58731,22.5984,175.638726,152.707705,823.928241,257.432377,47.223358,0.563481,23.387600,4.851915,0.023482,1.050225,0.069225,13.784111,1.302012,36.205956,69.08340,295.570575,0.23868,0.284232,89.245560,84.31664,29.657104,5.310690,1.74307,23.187704,7.294176,1.987283,1433.166750,0.949104,B,30.879420,78.526968,3.828384,13.394640,10.265073,9028.291921,3.583450,7.298162,1.73855,0.094822,11.339138,72.611063,2003.810319,22.136229,69.834944,0.120343,1
1,007255e47698,0.145282,978.76416,85.200147,36.968889,8.138688,3.632190,0.025578,13.517790,1.229900,5496.92824,19.4205,155.868030,14.754720,51.216883,257.432377,30.284345,0.484710,50.628208,6.085041,0.031442,1.113875,1.117800,28.310953,1.357182,37.476568,70.79836,178.553100,0.23868,0.363489,110.581815,75.74548,37.532000,0.005518,1.74307,17.222328,4.926396,0.858603,1111.287150,0.003042,A,109.125159,95.415086,52.260480,17.175984,0.296850,6785.003474,10.358927,0.173229,0.49706,0.568932,9.292698,72.611063,27981.562750,29.135430,32.131996,21.978000,0
2,013f2bd269f5,0.470030,2635.10654,85.200147,32.360553,8.138688,6.732840,0.025578,12.824570,1.229900,5135.78024,26.4825,128.988531,219.320160,482.141594,257.432377,32.563713,0.495852,85.955376,5.376488,0.036218,1.050225,0.700350,39.364743,1.009611,21.459644,70.81970,321.426625,0.23868,0.210441,120.056438,65.46984,28.053464,1.289739,1.74307,36.861352,7.813674,8.146651,1494.076488,0.377208,B,109.125159,78.526968,5.390628,224.207424,8.745201,8338.906181,11.626917,7.709560,0.97556,1.198821,37.077772,88.609437,13676.957810,28.022851,35.192676,0.196941,0
3,043ac50845d5,0.252107,3819.65177,120.201618,77.112203,8.138688,3.685344,0.025578,11.053708,1.229900,4169.67738,23.6577,237.282264,11.050410,661.518640,257.432377,15.201914,0.717882,88.159360,2.347652,0.029054,1.400300,0.636075,41.116960,0.722727,21.530392,47.27586,196.607985,0.23868,0.292431,139.824570,71.57120,24.354856,2.655345,1.74307,52.003884,7.386060,3.813326,15691.552180,0.614484,B,31.674357,78.526968,31.323372,59.301984,7.884336,10965.766040,14.852022,6.122162,0.49706,0.284466,18.529584,82.416803,2094.262452,39.948656,90.493248,0.155829,0
4,044fb8a146ec,0.380297,3733.04844,85.200147,14.103738,8.138688,3.942255,0.054810,3.396778,102.151980,5728.73412,24.0108,324.546318,149.717165,6074.859475,257.432377,82.213495,0.536467,72.644264,30.537722,0.025472,1.050225,0.693150,31.724726,0.827550,34.415360,74.06532,200.178160,0.23868,0.207708,97.920120,52.83888,26.019912,1.144902,1.74307,9.064856,7.350720,3.490846,1403.656300,0.164268,B,109.125159,91.994825,51.141336,29.102640,4.274640,16198.049590,13.666727,8.153058,48.50134,0.121914,16.408728,146.109943,8524.370502,45.381316,36.262628,0.096614,1


In [8]:
greeks_df.head()

,Id,Alpha,Beta,Gamma,Delta,Epsilon
0,000ff2bfdfe9,B,C,G,D,3/19/2019
1,007255e47698,A,C,M,B,Unknown
2,013f2bd269f5,A,C,M,B,Unknown
3,043ac50845d5,A,C,M,B,Unknown
4,044fb8a146ec,D,B,F,B,3/25/2020


In [9]:
list(set(train_df.Id)-set(greeks_df.Id)), list(set(greeks_df.Id)-set(train_df.Id))

([], [])

In [10]:
# [train.csv] The training set
#
# - Id 
#   : Unique identifier for each observation.
#
# - AB-GL 
#   : Fifty-six anonymized health characteristics.
#   : All are numeric except for EJ, which is categorical.
#
# - Class
#   : A binary target
#   : 1 indicates the subject has been diagnosed with one of the three conditions, 
#   : 0 indicates they have not.
#
# [greeks.csv] Supplemental metadata, only available for the training set.
#
# - Alpha
#   : Identifies the type of age-related condition, if present.
#   - A : No age-related condition. Corresponds to class 0
#   - B, D, G : The three age-related conditions. Correspond to class 1
# - Beta, Gamma, Delta
#   : Three experimental characteristics.
# - Epsilon
#   : The date the data for this subject was collected.
#   : Note that all of the data in the test set was collected after the training set was collected.

<br></br>

# Preprocess

<br>

## Add Interactions

In [11]:
import warnings
class AddInteraction:
    def __init__(self):
        warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
        
    def fit(self,features):
        self.features = features
        self.interaction_features = []
        for i in range(len(self.features)):
            for j in range(len(self.features)):
                if i>j:
                    col_i = self.features[i]
                    col_j = self.features[j]
                    self.interaction_features.append([col_i,col_j])
                    
    def transform(self,data):
        d = data.copy()
        for col_i,col_j in self.interaction_features:
            d[f'{col_i}*{col_j}'] = d[col_i]*d[col_j]
        return d

In [12]:
# define features type
target_feature = 'Class'
unuse_features = ['Id']
cat_features = ['EJ']
num_features = list(set(train_df.columns)-set([target_feature]+cat_features+unuse_features))

In [13]:
if CFG.ADD_INTERACTION:
    interaction_maker = AddInteraction()
    interaction_maker.fit(num_features)
    train_df2 = interaction_maker.transform(train_df)
    test_df2  = interaction_maker.transform(test_df)
else:
    train_df2 = train_df.copy()
    test_df2  = test_df.copy()

In [14]:
# define numerical type features
num_features = list(set(train_df2.columns)-set([target_feature]+cat_features+unuse_features))

<br>

## Feature Selection

### Numerical

In [15]:
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

In [16]:
def log_offset(x):
    if min(x)>0:
        offset = 0
    elif min(x)==0:
        offset = 1e-3
    else:
        offset = min(x)+1e-3
        print('minimum = {:.3f}'.format(min(x)))
    return np.log(x+offset)

In [17]:
CFG.FS_ALPHA

0.01

In [18]:
pvalue_list = []
for col in tqdm(num_features):
    ols_data = train_df2[[col,target_feature]].copy()
    ols_data.columns = ['feature','target']

    formula = 'feature ~ C(target)'
    model = ols(formula=formula,data=ols_data).fit()
    pvalue = anova_lm(model)['PR(>F)']['C(target)']
    
    pvalue_list.append([col,pvalue])
    
pvalue_df = pd.DataFrame(pvalue_list,columns=['feature','pvalue']).sort_values('pvalue',ascending=False)
unsignificant_features = pvalue_df[pvalue_df.pvalue>CFG.FS_ALPHA].feature.tolist()

100%|██████████| 55/55 [00:00<00:00, 135.84it/s]


In [19]:
log_pvalue_list = []
for col in tqdm(unsignificant_features):
    ols_data = train_df2[[col,target_feature]].copy()
    ols_data.columns = ['feature','target']
    ols_data['feature'] = log_offset(ols_data['feature'])

    formula = 'feature ~ C(target)'
    model = ols(formula=formula,data=ols_data).fit()
    pvalue = anova_lm(model)['PR(>F)']['C(target)']
    
    log_pvalue_list.append([col,pvalue])
    
log_pvalue_df = pd.DataFrame(log_pvalue_list,columns=['feature','pvalue']).sort_values('pvalue',ascending=False)
delete_features = log_pvalue_df[log_pvalue_df.pvalue>CFG.FS_ALPHA].feature.tolist()

100%|██████████| 29/29 [00:00<00:00, 124.36it/s]


In [20]:
train_df3 = train_df2.copy()
test_df3  = test_df2.copy()

train_df3.drop(columns=delete_features,inplace=True)
test_df3 .drop(columns=delete_features,inplace=True)

log_features = list(set(unsignificant_features)-set(delete_features))
for col in log_features:
    train_df3[col] = log_offset(train_df3[col])
    test_df3 [col] = log_offset(test_df3 [col])

In [21]:
print('> Log Features:', log_features)
print('> Delete Features:', delete_features)

> Log Features: ['AR', 'FI', 'CC', 'EB', 'DF']
> Delete Features: ['EG', 'AZ', 'FC', 'GH', 'BR', 'FS', 'CH', 'CB', 'AY', 'AH', 'CS', 'DN', 'GB', 'CW', 'CL', 'DV', 'EU', 'EL', 'AX', 'GI', 'EP', 'GE', 'CU', 'DY']


In [22]:
train_df2.shape,train_df3.shape

((617, 58), (617, 34))

In [23]:
# define features type
target_feature = 'Class'
unuse_features = ['Id']
cat_features = ['EJ']
num_features = list(set(train_df3.columns)-set([target_feature]+cat_features+unuse_features))

<br></br>

# Modeling

- No Interaction, Binary
- No Interaction, MultiClass
- Interaction, Binary
- Interaction, MultiClass

In [24]:
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import StratifiedKFold
import optuna
from optuna.samplers import TPESampler

In [25]:
def balanced_log_loss(y_true, y_pred):
    N_0 = np.sum(1 - y_true)
    N_1 = np.sum(y_true)
    p_1 = np.clip(y_pred, 1e-15, 1 - 1e-15)
    p_0 = 1 - p_1
    log_loss_0 = -np.sum((1 - y_true) * np.log(p_0))
    log_loss_1 = -np.sum(y_true * np.log(p_1))
    w_0 = 1 / N_0
    w_1 = 1 / N_1
    _balanced_log_loss = 2*(w_0 * log_loss_0 + w_1 * log_loss_1) / (w_0 + w_1)
    return _balanced_log_loss/(N_0+N_1)

In [28]:
fix_params_catboost = {
    'random_state':CFG.SEED,
    'early_stopping_rounds' : CFG.ES,
    'loss_function': 'Logloss',
    'grow_policy' : 'Lossguide', # 'SymmetricTree','Depthwise'
    'use_best_model' : True,
    'allow_writing_files' : False,
    'verbose' : False,
    'n_jobs' : -1,
}

def objective_catboost(trial):
    trial_params_catboost = {
        'learning_rate' : trial.suggest_float('learning_rate', 0.01, 0.2),
        'bagging_temperature' :trial.suggest_float('bagging_temperature', 0.01, 100.00),
        'iterations':trial.suggest_int('iterations', 3000, 5000),
        'max_depth':trial.suggest_int('max_depth', 4, 16),
        'random_strength' :trial.suggest_int('random_strength', 0, 100),
        'colsample_bylevel':trial.suggest_float('colsample_bylevel', 0.4, 1.0),
        'l2_leaf_reg':trial.suggest_float('l2_leaf_reg',1e-3,1),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'max_bin': trial.suggest_int('max_bin', 200, 500),
        'od_type': trial.suggest_categorical('od_type', ['IncToDec', 'Iter']),
        'auto_class_weights' : trial.suggest_categorical('auto_class_weights',['Balanced','None']),
    }
    # trial_params_catboost = {
    #     'learning_rate' : trial.suggest_float('learning_rate', 0.001, 0.03),
    #     'bagging_temperature' :trial.suggest_float('bagging_temperature', 0.01, 100.00),
    #     'iterations':trial.suggest_int('iterations', 20000, 50000),
    #     'max_depth':trial.suggest_int('max_depth', 4, 16),
    #     'random_strength' :trial.suggest_int('random_strength', 0, 100),
    #     'colsample_bylevel':trial.suggest_float('colsample_bylevel', 0.4, 1.0),
    #     'l2_leaf_reg':trial.suggest_float('l2_leaf_reg',1e-3,1),
    #     'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
    #     'max_bin': trial.suggest_int('max_bin', 200, 500),
    #     'od_type': trial.suggest_categorical('od_type', ['IncToDec', 'Iter']),
    #     'auto_class_weights' : trial.suggest_categorical('auto_class_weights',['Balanced','None']),
    # }
    params = {**fix_params_catboost,**trial_params_catboost}

    print('> Parameters:')
    for key,value in params.items():
        print('\t{}: {}'.format(key,value))
    
    skf = StratifiedKFold(n_splits=CFG.N_SPLITS,shuffle=True,random_state=CFG.SEED)
    models = []
    weights = []
    for train_idx,val_idx in skf.split(X,y):
        X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
        X_val  , y_val   = X.iloc[val_idx  ], y.iloc[val_idx  ]
        
        train_dataset = Pool(X_train,y_train,cat_features=cat_features)
        val_dataset   = Pool(X_val  ,y_val  ,cat_features=cat_features)
        
        model = CatBoostClassifier(**params)
        model.fit(
            train_dataset,
            eval_set=val_dataset,
        )
        loss = balanced_log_loss(y_val.values, model.predict_proba(X_val)[:, 1])
        
        models.append(model)
        weights.append(1/loss)

    weights /= sum(weights)
    dataset = Pool(X,y,cat_features=cat_features)

    y_probas = np.zeros((len(X),2))
    for weight,model in zip(weights,models):
        y_probas[:, 0] += weight * model.predict_proba(dataset)[:, 0]
        y_probas[:, 1] += weight * model.predict_proba(dataset)[:, 1]
    y_probas /= sum(weights)
    
    score = balanced_log_loss(y_true=y.values,y_pred=y_probas[:,1])
    #print('> Balanced Logloss: {:.4f}'.format(score))

    return score

In [29]:
%%time

X = train_df3[cat_features+num_features]
y = train_df3[target_feature]

sampler = TPESampler(seed=CFG.SEED)
study = optuna.create_study(
    study_name = 'catboostclassifier_optuna',
    direction = 'minimize',
    sampler = sampler,
)
study.optimize(objective, n_trials=CFG.N_TRIALS, n_jobs=CFG.N_JOBS)
print("Best Score:",study.best_value)
print("Best trial",study.best_trial.params)

[I 2023-06-01 21:43:23,298] A new study created in memory with name: catboostclassifier_optuna


> Parameters:
	random_state: 0
	early_stopping_rounds: 300
	loss_function: Logloss
	grow_policy: Lossguide
	use_best_model: True
	allow_writing_files: False
	verbose: False
	learning_rate: 0.1142745657461917
	bagging_temperature: 71.52178474357822
	iterations: 4206
	max_depth: 11
	random_strength: 42
	colsample_bylevel: 0.7875364678399936
	l2_leaf_reg: 0.4381496240514298
	min_child_samples: 90
	max_bin: 490
	od_type: Iter
	auto_class_weights: None


[I 2023-06-01 21:43:28,321] Trial 0 finished with value: 0.06995241230833286 and parameters: {'learning_rate': 0.1142745657461917, 'bagging_temperature': 71.52178474357822, 'iterations': 4206, 'max_depth': 11, 'random_strength': 42, 'colsample_bylevel': 0.7875364678399936, 'l2_leaf_reg': 0.4381496240514298, 'min_child_samples': 90, 'max_bin': 490, 'od_type': 'Iter', 'auto_class_weights': 'None'}. Best is trial 0 with value: 0.06995241230833286.


> Parameters:
	random_state: 0
	early_stopping_rounds: 300
	loss_function: Logloss
	grow_policy: Lossguide
	use_best_model: True
	allow_writing_files: False
	verbose: False
	learning_rate: 0.1858633612756056
	bagging_temperature: 7.112895459206714
	iterations: 3174
	max_depth: 4
	random_strength: 84
	colsample_bylevel: 0.8668940505699103
	l2_leaf_reg: 0.8701421360985724
	min_child_samples: 98
	max_bin: 440
	od_type: Iter
	auto_class_weights: None


[I 2023-06-01 21:43:30,667] Trial 1 finished with value: 0.07323634677200146 and parameters: {'learning_rate': 0.1858633612756056, 'bagging_temperature': 7.112895459206714, 'iterations': 3174, 'max_depth': 4, 'random_strength': 84, 'colsample_bylevel': 0.8668940505699103, 'l2_leaf_reg': 0.8701421360985724, 'min_child_samples': 98, 'max_bin': 440, 'od_type': 'Iter', 'auto_class_weights': 'None'}. Best is trial 0 with value: 0.06995241230833286.


> Parameters:
	random_state: 0
	early_stopping_rounds: 300
	loss_function: Logloss
	grow_policy: Lossguide
	use_best_model: True
	allow_writing_files: False
	verbose: False
	learning_rate: 0.037237124607718816
	bagging_temperature: 94.46744501578789
	iterations: 4044
	max_depth: 9
	random_strength: 26
	colsample_bylevel: 0.86454021366053
	l2_leaf_reg: 0.456694181884332
	min_child_samples: 59
	max_bin: 205
	od_type: IncToDec
	auto_class_weights: None


[W 2023-06-01 21:43:35,293] Trial 2 failed with parameters: {'learning_rate': 0.037237124607718816, 'bagging_temperature': 94.46744501578789, 'iterations': 4044, 'max_depth': 9, 'random_strength': 26, 'colsample_bylevel': 0.86454021366053, 'l2_leaf_reg': 0.456694181884332, 'min_child_samples': 59, 'max_bin': 205, 'od_type': 'IncToDec', 'auto_class_weights': 'None'} because of the following error: KeyboardInterrupt('').
Traceback (most recent call last):
  File "/Users/khj/.pyenv/versions/3.8.10/lib/python3.8/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "<ipython-input-26-a10e49060a50>", line 55, in objective
    model.fit(
  File "/Users/khj/.pyenv/versions/3.8.10/lib/python3.8/site-packages/catboost/core.py", line 5131, in fit
    self._fit(X, y, cat_features, text_features, embedding_features, None, sample_weight, None, None, None, None, baseline, use_best_model,
  File "/Users/khj/.pyenv/versions/3.8.10/lib/python3.8/site

KeyboardInterrupt: 

<br></br>

# Inference

In [ ]:
best_params = {**fix_params,**study.best_params}

In [ ]:
%%time

skf = StratifiedKFold(n_splits=CFG.N_SPLITS,shuffle=True,random_state=CFG.SEED)

X = train_df3[cat_features+num_features]
y = train_df3[target_feature]

models = []
weights = []
for train_idx,val_idx in tqdm(skf.split(X,y),total=CFG.N_SPLITS):
    X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
    X_val  , y_val   = X.iloc[val_idx  ], y.iloc[val_idx  ]

    train_dataset = Pool(X_train,y_train,cat_features=cat_features)
    val_dataset   = Pool(X_val  ,y_val  ,cat_features=cat_features)

    model = CatBoostClassifier(**best_params)
    model.fit(
        train_dataset,
        eval_set=val_dataset,
    )
    loss = balanced_log_loss(y_val.values, model.predict_proba(X_val)[:, 1])

    models.append(model)
    weights.append(1/loss)

In [ ]:
weights /= sum(weights)

<br>

## Train

In [ ]:
dataset = Pool(X,y,cat_features=cat_features)

y_probas = np.zeros((len(X),2))
for weight,model in zip(weights,models):
    y_probas[:, 0] += weight * model.predict_proba(dataset)[:, 0]
    y_probas[:, 1] += weight * model.predict_proba(dataset)[:, 1]
y_probas /= sum(weights)
y_probas

In [ ]:
balanced_log_loss(y_true=y,y_pred=y_probas[:,1])

In [ ]:
pd.crosstab(y,[0 if p>q else 1 for p,q in y_probas])

<br>

## Test

In [ ]:
dataset = Pool(test_df3[cat_features+num_features],cat_features=cat_features)

y_probas = np.zeros((len(test_df3),2))
for weight,model in zip(weights,models):
    y_probas[:, 0] += weight * model.predict_proba(dataset)[:, 0]
    y_probas[:, 1] += weight * model.predict_proba(dataset)[:, 1]
y_probas /= sum(weights)
y_probas

In [ ]:
# submit = pd.read_csv('./data/sample_submission.csv')
submit = pd.read_csv(CFG.REF_PATH+'sample_submission.csv')
submit['class_0'] = y_probas[:,0]
submit['class_1'] = y_probas[:,1]

In [ ]:
submit.to_csv('/kaggle/working/submission.csv', index=False)